In [1]:
import json
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
file = "../input/metrics_dataset-domesticDeclarations.json"
with open(file) as f:
    metrics = json.load(f)

c:\Users\maribel\AppData\Local\Anaconda3\envs\dialogact\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
phrases =  [metric for metric in metrics["metrics"]]

In [3]:
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-large")
model = AutoModelForSeq2SeqLM.from_pretrained("./models/GeneralParser_flant5")

In [4]:
for input_string in phrases:
    if input_string["type"] != "count":
        continue
    available_tags = ['TSE', 'TEE', 'TBE', 'CE','AttributeName', 'AttributeValue', 'AGR', 'GBC', 'FDE']
    prompt = f"Sentence: {input_string['description']}\nAvailable Tags: {', '.join(available_tags)}"
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids
    outputs = model.generate(input_ids, max_length=1000)
    parsing_result = tokenizer.decode(outputs[0], skip_special_tokens=True)
    print("Input:", input_string['description'])
    print("Expected:", input_string["slots"])
    print("Predicted:", parsing_result)
    print("")

Input: Total workflow rejections by approver
Expected: {'AGR': 'Total', 'CE': 'workflow rejections by approver'}
Predicted: Total: AGR; workflow rejections: CE; approver: GBC

Input: percentage of rejections by approver
Expected: {'AGR': 'percentage of', 'CE': 'rejections by approver'}
Predicted: percentage of: AGR; rejections: CE; approver: FDE

Input: Total workflow rejected by worker
Expected: {'AGR': 'Total', 'CE': 'workflow rejected by worker'}
Predicted: Total: AGR; workflow rejected: CE; worker: GBC

Input: percentage of rejected workflows by worker
Expected: {'AGR': 'percentage of', 'CE': 'rejected workflows by worker'}
Predicted: rejected workflows by worker: CE

Input: The % of requests the director approves
Expected: {'CE': 'requests the director approves', 'AGR': '% of'}
Predicted: AGR; requests the director approves: CE

Input: The % of submitted declarations that are rejected
Expected: {'AGR': '% of', 'CE': 'submitted declarations that are rejected'}
Predicted: AGR; submi